In [1]:
"""
项目：深度学习方法+机器学习方法在dga域名区分中的应用。
"""

'\n项目：深度学习方法+机器学习方法在dga域名区分中的应用。\n'

In [1]:
#DGA域名识别
#1:导入相关的包
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
import lightgbm as lgb
from sklearn import preprocessing
from hmmlearn import hmm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell

dga_file="E:/pycharm_project/deep_learning_web_safe/dga/data/dga.txt"
alexa_file="E:/pycharm_project/deep_learning_web_safe/dga/data/dga1m.csv"

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


D:\anaconda3_5\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
D:\anaconda3_5\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#2:数据读取及特征提取
#正样本
def load_alexa():
    x=[]
    data = pd.read_csv(alexa_file, sep=",",header=None)
    x=[i[1] for i in data.values]
    return x

#负样本
def load_dga():
    x=[]
    data = pd.read_csv(dga_file, sep="\t", header=None)
    x=[i[1] for i in data.values]
    return x

***通过2gram获取文本特征

In [6]:
def get_feature_2gram():
    alexa=load_alexa()
    dga=load_dga()
    x=alexa+dga
    max_features=10000
    y=[0]*len(alexa)+[1]*len(dga)

    CV = CountVectorizer(
                                    ngram_range=(2, 2),
                                    token_pattern=r'\w',
                                    decode_error='ignore',
                                    strip_accents='ascii',
                                    max_features=max_features,
                                    stop_words='english',
                                    max_df=1.0,
                                    min_df=1)
    x = CV.fit_transform(x)
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4)

    return x_train.toarray(), x_test.toarray(), y_train, y_test

x_train, x_test, y_train, y_test = get_feature_2gram()
x_train#(11989,1156)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

***通过TF-IDF提取文本特征

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_feature_tfidf():
    alexa=load_alexa()
    dga=load_dga()
    x=alexa+dga
    max_features=10000
    y=[0]*len(alexa)+[1]*len(dga)
    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='ascii',
        token_pattern=r'\w', 
        decode_error='ignore',
        stop_words='english',
        ngram_range=(1, 1),
        max_features=10000)
    word_vectorizer.fit(x) #这里确实是拿了全部的样本的
    train_word_features = word_vectorizer.transform(x)
    
    x_train, x_test, y_train, y_test=train_test_split(train_word_features,y,test_size=0.4)
    return x_train.toarray(), x_test.toarray(), y_train, y_test
    # return train_word_features
    # x = word_vectorizer.fit_transform(x)
    # x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4)
x_train, x_test, y_train, y_test = get_feature_tfidf()
x_train 

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.63758948, 0.        , ..., 0.15713093, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.53706537, 0.        , ..., 0.24505601, 0.13960737,
        0.29971481]])

***通过234_gram获取文本特征

In [3]:
def get_feature_234gram():
    alexa=load_alexa()
    dga=load_dga()
    x=alexa+dga
    max_features=10000
    y=[0]*len(alexa)+[1]*len(dga)

    CV = CountVectorizer(
                                    ngram_range=(2, 4),
                                    token_pattern=r'\w',
                                    decode_error='ignore',
                                    strip_accents='ascii',
                                    max_features=max_features,
                                    stop_words='english',
                                    max_df=1.0,
                                    min_df=1)
    x = CV.fit_transform(x)
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4)

    return x_train.toarray(), x_test.toarray(), y_train, y_test

x_train, x_test, y_train, y_test = get_feature_234gram()
x_train

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [4]:
"""
1:域名中元音字母的个数
2:去重以后元音字母的个数
3:域名中数字的个数
"""
def get_aeiou(domain):
    count = len(re.findall(r'[aeiou]', domain.lower()))
    #count = (0.0 + count) / len(domain)
    return count

def get_uniq_char_num(domain):
    count=len(set(domain))
    #count=(0.0+count)/len(domain)
    return count

def get_uniq_num_num(domain):
    count = len(re.findall(r'[1234567890]', domain.lower()))
    #count = (0.0 + count) / len(domain)
    return count

def get_feature():
    from sklearn import preprocessing
    alexa=load_alexa()
    dga=load_dga()
    v=alexa+dga
    y=[0]*len(alexa)+[1]*len(dga)
    x=[]

    for vv in v:
        vvv=[get_aeiou(vv),get_uniq_char_num(vv),get_uniq_num_num(vv),len(vv)]
        x.append(vvv)

    x=preprocessing.scale(x) #标准化处理
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_feature()
x_train

array([[-0.44192681, -0.44017182, -0.25943017, -0.55170028],
       [-0.44192681,  1.63785621, -0.25943017,  1.90731546],
       [-0.44192681, -0.69992533, -0.25943017, -0.70538876],
       ...,
       [-1.5419681 , -0.95967883, -0.25943017, -1.01276573],
       [-0.99194746, -0.69992533, -0.25943017, -0.39801179],
       [ 1.20813513, -0.18041832, -0.25943017,  0.06305366]])

In [11]:
"""
字符序列模型,字符组成的序列，字符转换成对应的ASCII值，这样就可以把域名最终转换成一个数字序列：可以理解为是具有时序特征的时序序列
"""
def get_feature_charseq():
    alexa=load_alexa()
    dga=load_dga()
    x=alexa+dga
    max_features=10000
    y=[0]*len(alexa)+[1]*len(dga)

    t=[]
    for i in x:
        v=[]
        for j in range(0,len(i)):
            v.append(ord(i[j]))
        t.append(v)

    x=t
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4)

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_feature_charseq()
x_train[20:25]

[[98, 101, 104, 105, 110, 100, 119, 111, 111, 100, 115, 46, 99, 111, 109],
 [111, 118, 101, 114, 45, 98, 108, 111, 103, 46, 99, 111, 109],
 [99, 114, 97, 122, 121, 103, 97, 109, 101, 115, 46, 99, 111, 109],
 [97, 116, 108, 97, 115, 115, 105, 97, 110, 46, 99, 111, 109],
 [98, 105, 110, 116, 106, 98, 101, 105, 108, 46, 111, 114, 103]]

***接下来开始定义训练模型

In [7]:
#3：朴素贝叶斯模型
def do_nb(x_train, x_test, y_train, y_test):
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(classification_report(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))

def do_xgboost(x_train, x_test, y_train, y_test):
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))

#调参主要是体现在num_class,以及metri两个参数取值的差异上。    
def do_lightgbm(x_train, x_test, y_train, y_test):
    lgb_train = lgb.Dataset(x_train,y_train)
    lgb_test = lgb.Dataset(x_test,y_test,reference=lgb_train)
    gbm_parm = {"nthread":5,
           "num_class": 1,  
           "objective": 'regression',
           "learning_rate":0.01,
           "num_leaves":64,
           "bagging_fraction":0.9,
           "feature_fraction":0.7,
           "lambda_l1":0.001,
           "lambda_l2":0,
           "bagging_seed":200,
           'metric': 'rmse'     
           }
    evals_result = {}
    
    gbm = lgb.train(gbm_parm,lgb_train,
               num_boost_round=20000,
               valid_sets=[lgb_train,lgb_test],
               early_stopping_rounds=100, #如果超过100次效率没有得到提升
               evals_result=evals_result,
                verbose_eval=10)
    
    y_pred = gbm.predict(x_test,num_iteration=gbm.best_iteration)
    y_pred =[1 if i>0.5 else 0 for i in y_pred]
    print(classification_report(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))

def do_mlp(x_train, x_test, y_train, y_test):

    global max_features
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))

def do_rnn(trainX, testX, trainY, testY):
    max_document_length=32
    y_test=testY
    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)  #不到最大长度的数据填充0
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)       #把标记数据进行二值化处理
    testY = to_categorical(testY, nb_classes=2)

    # Network building
    net = tflearn.input_data([None, max_document_length])  #定义RNN结构，使用最简单的单层LSTM结构
    net = tflearn.embedding(net, input_dim=10240000, output_dim=32)  #看看试着把维度降低效率是否会好点
    net = tflearn.lstm(net, 32, dropout=0.1)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='categorical_crossentropy')

    # Training
    model = tflearn.DNN(net, tensorboard_verbose=0,tensorboard_dir="dga_log")
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True,
              batch_size=10,run_id="dga",n_epoch=1)

    y_predict_list = model.predict(testX)
    #print y_predict_list

    y_predict = []
    for i in y_predict_list:
        print  (i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    print(classification_report(y_test, y_predict))
    print (metrics.confusion_matrix(y_test, y_predict))

In [72]:
"""
1.1:   2gram模型
"""
x_train, x_test, y_train, y_test = get_feature_2gram()
print("start nb ----->")
do_nb(x_train, x_test, y_train, y_test)
print("start xgboost ----->")
do_xgboost(x_train, x_test, y_train, y_test)
print("start mlp ----->")
do_mlp(x_train, x_test, y_train, y_test)
#print("start rnn ----->")
#do_rnn(x_train, x_test, y_train, y_test)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start nb ----->
              precision    recall  f1-score   support

           0       0.73      0.96      0.83      4032
           1       0.94      0.64      0.77      3961

    accuracy                           0.80      7993
   macro avg       0.84      0.80      0.80      7993
weighted avg       0.84      0.80      0.80      7993

[[3882  150]
 [1409 2552]]
start xgboost ----->


D:\anaconda3_5\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:19:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      4032
           1       0.95      0.92      0.94      3961

    accuracy                           0.94      7993
   macro avg       0.94      0.94      0.94      7993
weighted avg       0.94      0.94      0.94      7993

[[3850  182]
 [ 306 3655]]
start mlp ----->
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4032
           1       0.96      0.96      0.96      3961

    accuracy                           0.96      7993
   macro avg       0.96      0.96      0.96      7993
weighted avg       0.96      0.96      0.96    

D:\anaconda3_5\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [69]:
"""
1.2：  2gram模型
"""
x_train, x_test, y_train, y_test = get_feature_2gram()
print("start lightgbm ---->")
do_lightgbm(x_train, x_test, y_train, y_test)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start lightgbm ---->


D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:261: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2101
[LightGBM] [Info] Number of data points in the train set: 11989, number of used features: 761
[LightGBM] [Info] Start training from score 0.498040
Training until validation scores don't improve for 100 rounds
[10]	training's rmse: 0.476647	valid_1's rmse: 0.477587
[20]	training's rmse: 0.455673	valid_1's rmse: 0.457569
[30]	training's rmse: 0.436911	valid_1's rmse: 0.439626
[40]	training's rmse: 0.420433	valid_1's rmse: 0.424028
[50]	training's rmse: 0.405257	valid_1's rmse: 0.409866
[60]	training's rmse: 0.391882	valid_1's rmse: 0.397452
[70]	training's rmse: 0.379867	valid_1's rmse: 0.386491
[80]	training's rmse: 0.369093	valid_1's rmse: 0.37667
[90]	training's rmse: 0.358844	valid_1's rmse: 0.367462
[100]	training's rmse: 0.349957	valid_1's rmse: 0.359541
[110]	training's rmse: 0.341647	

[1420]	training's rmse: 0.17756	valid_1's rmse: 0.232061
[1430]	training's rmse: 0.177346	valid_1's rmse: 0.232001
[1440]	training's rmse: 0.177116	valid_1's rmse: 0.231942
[1450]	training's rmse: 0.176918	valid_1's rmse: 0.231893
[1460]	training's rmse: 0.176685	valid_1's rmse: 0.231813
[1470]	training's rmse: 0.176456	valid_1's rmse: 0.231749
[1480]	training's rmse: 0.176231	valid_1's rmse: 0.231671
[1490]	training's rmse: 0.176022	valid_1's rmse: 0.231629
[1500]	training's rmse: 0.175798	valid_1's rmse: 0.231583
[1510]	training's rmse: 0.175563	valid_1's rmse: 0.231517
[1520]	training's rmse: 0.175352	valid_1's rmse: 0.231474
[1530]	training's rmse: 0.175133	valid_1's rmse: 0.231412
[1540]	training's rmse: 0.174919	valid_1's rmse: 0.23136
[1550]	training's rmse: 0.174702	valid_1's rmse: 0.231286
[1560]	training's rmse: 0.174479	valid_1's rmse: 0.231226
[1570]	training's rmse: 0.174265	valid_1's rmse: 0.231155
[1580]	training's rmse: 0.174073	valid_1's rmse: 0.231115
[1590]	training'

[2850]	training's rmse: 0.152222	valid_1's rmse: 0.225735
[2860]	training's rmse: 0.152069	valid_1's rmse: 0.225691
[2870]	training's rmse: 0.151928	valid_1's rmse: 0.225668
[2880]	training's rmse: 0.151786	valid_1's rmse: 0.225627
[2890]	training's rmse: 0.151646	valid_1's rmse: 0.225601
[2900]	training's rmse: 0.151509	valid_1's rmse: 0.225567
[2910]	training's rmse: 0.151366	valid_1's rmse: 0.225531
[2920]	training's rmse: 0.151228	valid_1's rmse: 0.225505
[2930]	training's rmse: 0.151084	valid_1's rmse: 0.225482
[2940]	training's rmse: 0.150945	valid_1's rmse: 0.225454
[2950]	training's rmse: 0.150797	valid_1's rmse: 0.225428
[2960]	training's rmse: 0.15066	valid_1's rmse: 0.225385
[2970]	training's rmse: 0.150527	valid_1's rmse: 0.225358
[2980]	training's rmse: 0.150386	valid_1's rmse: 0.225335
[2990]	training's rmse: 0.15023	valid_1's rmse: 0.225305
[3000]	training's rmse: 0.150091	valid_1's rmse: 0.22527
[3010]	training's rmse: 0.149951	valid_1's rmse: 0.225233
[3020]	training's

[4310]	training's rmse: 0.13467	valid_1's rmse: 0.222632
[4320]	training's rmse: 0.134573	valid_1's rmse: 0.222619
[4330]	training's rmse: 0.134469	valid_1's rmse: 0.222606
[4340]	training's rmse: 0.134371	valid_1's rmse: 0.222593
[4350]	training's rmse: 0.134277	valid_1's rmse: 0.222578
[4360]	training's rmse: 0.134174	valid_1's rmse: 0.222564
[4370]	training's rmse: 0.134076	valid_1's rmse: 0.222551
[4380]	training's rmse: 0.133982	valid_1's rmse: 0.222536
[4390]	training's rmse: 0.133881	valid_1's rmse: 0.222531
[4400]	training's rmse: 0.133784	valid_1's rmse: 0.222521
[4410]	training's rmse: 0.133684	valid_1's rmse: 0.222504
[4420]	training's rmse: 0.133586	valid_1's rmse: 0.22249
[4430]	training's rmse: 0.133491	valid_1's rmse: 0.222473
[4440]	training's rmse: 0.133395	valid_1's rmse: 0.222457
[4450]	training's rmse: 0.133298	valid_1's rmse: 0.22244
[4460]	training's rmse: 0.133203	valid_1's rmse: 0.222431
[4470]	training's rmse: 0.133104	valid_1's rmse: 0.22241
[4480]	training's 

[7190]	training's rmse: 0.111698	valid_1's rmse: 0.220362
[7200]	training's rmse: 0.111638	valid_1's rmse: 0.22036
[7210]	training's rmse: 0.111574	valid_1's rmse: 0.220357
[7220]	training's rmse: 0.111508	valid_1's rmse: 0.220353
[7230]	training's rmse: 0.111445	valid_1's rmse: 0.22035
[7240]	training's rmse: 0.111381	valid_1's rmse: 0.220349
[7250]	training's rmse: 0.111313	valid_1's rmse: 0.220348
[7260]	training's rmse: 0.111252	valid_1's rmse: 0.220351
[7270]	training's rmse: 0.111186	valid_1's rmse: 0.220355
[7280]	training's rmse: 0.111123	valid_1's rmse: 0.220346
[7290]	training's rmse: 0.11106	valid_1's rmse: 0.220336
[7300]	training's rmse: 0.111002	valid_1's rmse: 0.220331
[7310]	training's rmse: 0.110944	valid_1's rmse: 0.220329
[7320]	training's rmse: 0.110881	valid_1's rmse: 0.220328
[7330]	training's rmse: 0.110824	valid_1's rmse: 0.220325
[7340]	training's rmse: 0.110763	valid_1's rmse: 0.220323
[7350]	training's rmse: 0.110697	valid_1's rmse: 0.22032
[7360]	training's 

[8640]	training's rmse: 0.10317	valid_1's rmse: 0.220027
[8650]	training's rmse: 0.103118	valid_1's rmse: 0.22003
[8660]	training's rmse: 0.103064	valid_1's rmse: 0.220025
[8670]	training's rmse: 0.103004	valid_1's rmse: 0.220018
[8680]	training's rmse: 0.102956	valid_1's rmse: 0.220018
[8690]	training's rmse: 0.102905	valid_1's rmse: 0.22002
[8700]	training's rmse: 0.10285	valid_1's rmse: 0.220014
[8710]	training's rmse: 0.102797	valid_1's rmse: 0.220009
[8720]	training's rmse: 0.102742	valid_1's rmse: 0.220001
[8730]	training's rmse: 0.102692	valid_1's rmse: 0.220004
[8740]	training's rmse: 0.102641	valid_1's rmse: 0.22001
[8750]	training's rmse: 0.10259	valid_1's rmse: 0.220008
[8760]	training's rmse: 0.10254	valid_1's rmse: 0.220008
[8770]	training's rmse: 0.10249	valid_1's rmse: 0.220005
[8780]	training's rmse: 0.102438	valid_1's rmse: 0.220002
[8790]	training's rmse: 0.102389	valid_1's rmse: 0.22
[8800]	training's rmse: 0.102335	valid_1's rmse: 0.220001
[8810]	training's rmse: 0.

In [70]:
"""
2.1：  TF-IDF模型
"""
x_train, x_test, y_train, y_test = get_feature_tfidf()
print("start nb ----->")
do_nb(x_train, x_test, y_train, y_test)
print("start xgboost ----->")
do_xgboost(x_train, x_test, y_train, y_test)
print("start mlp ----->")
do_mlp(x_train, x_test, y_train, y_test)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start nb ----->
              precision    recall  f1-score   support

           0       0.77      0.90      0.83      3997
           1       0.88      0.73      0.80      3996

    accuracy                           0.82      7993
   macro avg       0.82      0.82      0.81      7993
weighted avg       0.82      0.82      0.81      7993

[[3586  411]
 [1060 2936]]
start xgboost ----->
[15:13:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\anaconda3_5\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3997
           1       0.95      0.93      0.94      3996

    accuracy                           0.94      7993
   macro avg       0.94      0.94      0.94      7993
weighted avg       0.94      0.94      0.94      7993

[[3802  195]
 [ 293 3703]]
start mlp ----->
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      3997
           1       0.93      0.91      0.92      3996

    accuracy                           0.92      7993
   macro avg       0.92      0.92      0.92      7993
weighted avg       0.92      0.92      0.92      7993

[[3740  257]
 [ 351 3645]]


In [71]:
"""
2.2：  TF-IDF模型
"""
x_train, x_test, y_train, y_test = get_feature_tfidf()
print("start lightgbm ---->")
do_lightgbm(x_train, x_test, y_train, y_test)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start lightgbm ---->
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001500 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7772
[LightGBM] [Info] Number of data points in the train set: 11989, number of used features: 34
[LightGBM] [Info] Start training from score 0.504462
Training until validation scores don't improve for 100 rounds
[10]	training's rmse: 0.468868	valid_1's rmse: 0.469861
[20]	training's rmse: 0.440579	valid_1's rmse: 0.44275


D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:261: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in 

[30]	training's rmse: 0.415144	valid_1's rmse: 0.41836
[40]	training's rmse: 0.392268	valid_1's rmse: 0.396618
[50]	training's rmse: 0.371794	valid_1's rmse: 0.377367
[60]	training's rmse: 0.353943	valid_1's rmse: 0.360799
[70]	training's rmse: 0.337188	valid_1's rmse: 0.345314
[80]	training's rmse: 0.322153	valid_1's rmse: 0.331624
[90]	training's rmse: 0.308871	valid_1's rmse: 0.319664
[100]	training's rmse: 0.29699	valid_1's rmse: 0.309088
[110]	training's rmse: 0.286229	valid_1's rmse: 0.299718
[120]	training's rmse: 0.276905	valid_1's rmse: 0.291713
[130]	training's rmse: 0.26859	valid_1's rmse: 0.284631
[140]	training's rmse: 0.261099	valid_1's rmse: 0.278391
[150]	training's rmse: 0.254313	valid_1's rmse: 0.272851
[160]	training's rmse: 0.248381	valid_1's rmse: 0.268097
[170]	training's rmse: 0.242983	valid_1's rmse: 0.263897
[180]	training's rmse: 0.238207	valid_1's rmse: 0.260163
[190]	training's rmse: 0.233818	valid_1's rmse: 0.256943
[200]	training's rmse: 0.229912	valid_1's

In [73]:
"""
3.1：  统计特征模型
"""
x_train, x_test, y_train, y_test = get_feature()
print("start nb ----->")
do_nb(x_train, x_test, y_train, y_test)
print("start xgboost ----->")
do_xgboost(x_train, x_test, y_train, y_test)
print("start mlp ----->")
do_mlp(x_train, x_test, y_train, y_test)

start nb ----->
              precision    recall  f1-score   support

           0       0.65      0.90      0.76      4014
           1       0.84      0.52      0.64      3979

    accuracy                           0.71      7993
   macro avg       0.75      0.71      0.70      7993
weighted avg       0.75      0.71      0.70      7993

[[3628  386]
 [1912 2067]]
start xgboost ----->
[15:33:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\anaconda3_5\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.83      0.92      0.87      4014
           1       0.91      0.80      0.85      3979

    accuracy                           0.86      7993
   macro avg       0.87      0.86      0.86      7993
weighted avg       0.87      0.86      0.86      7993

[[3694  320]
 [ 779 3200]]
start mlp ----->
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      4014
           1       0.92      0.80      0.85      3979

    accuracy                           0.86      7993
   macro avg       0.87      0.86      0.86      7993
weighted avg       0.87      0.86      0.86      7993

[[3720  294]
 [ 809 3170]]


D:\anaconda3_5\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [74]:
"""
3.2:  统计特征模型
"""
x_train, x_test, y_train, y_test = get_feature()
print("start lightgbm ---->")
do_lightgbm(x_train, x_test, y_train, y_test)

start lightgbm ---->
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98
[LightGBM] [Info] Number of data points in the train set: 11989, number of used features: 4
[LightGBM] [Info] Start training from score 0.496372
Training until validation scores don't improve for 100 rounds
[10]	training's rmse: 0.478133	valid_1's rmse: 0.478455
[20]	training's rmse: 0.45526	valid_1's rmse: 0.45586
[30]	training's rmse: 0.437936	valid_1's rmse: 0.438835
[40]	training's rmse: 0.420834	valid_1's rmse: 0.422057
[50]	training's rmse: 0.405881	valid_1's rmse: 0.407399
[60]	training's rmse: 0.392453	valid_1's rmse: 0.394341
[70]	training's rmse: 0.382488	valid_1's rmse: 0.384722
[80]	training's rmse: 0.371578	valid_1's rmse: 0.374252
[90]	training's rmse: 0.36339	valid_1's rmse: 0.366358
[100]	trai

D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:261: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in 


[130]	training's rmse: 0.338343	valid_1's rmse: 0.342464
[140]	training's rmse: 0.333891	valid_1's rmse: 0.338309
[150]	training's rmse: 0.330261	valid_1's rmse: 0.334983
[160]	training's rmse: 0.327219	valid_1's rmse: 0.332158
[170]	training's rmse: 0.325046	valid_1's rmse: 0.330178
[180]	training's rmse: 0.323052	valid_1's rmse: 0.328383
[190]	training's rmse: 0.321259	valid_1's rmse: 0.326816
[200]	training's rmse: 0.319574	valid_1's rmse: 0.325337
[210]	training's rmse: 0.318229	valid_1's rmse: 0.324239
[220]	training's rmse: 0.317038	valid_1's rmse: 0.323228
[230]	training's rmse: 0.316187	valid_1's rmse: 0.322524
[240]	training's rmse: 0.315296	valid_1's rmse: 0.321804
[250]	training's rmse: 0.314347	valid_1's rmse: 0.321043
[260]	training's rmse: 0.31355	valid_1's rmse: 0.320417
[270]	training's rmse: 0.31298	valid_1's rmse: 0.319988
[280]	training's rmse: 0.312412	valid_1's rmse: 0.319577
[290]	training's rmse: 0.311895	valid_1's rmse: 0.319209
[300]	training's rmse: 0.311398	

In [5]:
"""
4.1: 2_3_4gram模型
"""
x_train, x_test, y_train, y_test = get_feature_234gram()
print("start nb ----->")
do_nb(x_train, x_test, y_train, y_test)
print("start xgboost ----->")
do_xgboost(x_train, x_test, y_train, y_test)
print("start mlp ----->")
do_mlp(x_train, x_test, y_train, y_test)


D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start nb ----->
              precision    recall  f1-score   support

           0       0.93      0.88      0.91      3956
           1       0.89      0.94      0.91      4037

    accuracy                           0.91      7993
   macro avg       0.91      0.91      0.91      7993
weighted avg       0.91      0.91      0.91      7993

[[3497  459]
 [ 255 3782]]
start xgboost ----->


D:\anaconda3_5\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:02:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      3956
           1       0.96      0.93      0.94      4037

    accuracy                           0.94      7993
   macro avg       0.94      0.94      0.94      7993
weighted avg       0.94      0.94      0.94      7993

[[3780  176]
 [ 300 3737]]
start mlp ----->
              precision    recall  f1-score   support

           0       0.49      1.00      0.66      3956
           1       0.00      0.00      0.00      4037

    accuracy                           0.49      7993
   macro avg       0.25      0.50      0.33      7993
weighted avg       0.24      0.49      0.33    

D:\anaconda3_5\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3_5\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3_5\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
"""
4.2: 2_3_4gram模型
"""
x_train, x_test, y_train, y_test = get_feature_234gram()
print("start lightgbm ---->")
do_lightgbm(x_train, x_test, y_train, y_test)

D:\anaconda3_5\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


start lightgbm ---->


D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
D:\anaconda3_5\lib\site-packages\lightgbm\engine.py:261: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3986
[LightGBM] [Info] Number of data points in the train set: 11989, number of used features: 1665
[LightGBM] [Info] Start training from score 0.499041
Training until validation scores don't improve for 100 rounds
[10]	training's rmse: 0.475989	valid_1's rmse: 0.477271
[20]	training's rmse: 0.454153	valid_1's rmse: 0.456755
[30]	training's rmse: 0.435033	valid_1's rmse: 0.43893
[40]	training's rmse: 0.41784	valid_1's rmse: 0.422979
[50]	training's rmse: 0.402718	valid_1's rmse: 0.409024
[60]	training's rmse: 0.389093	valid_1's rmse: 0.396515
[70]	training's rmse: 0.376848	valid_1's rmse: 0.385401
[80]	training's rmse: 0.365795	valid_1's rmse: 0.375469
[90]	training's rmse: 0.355781	valid_1's rmse: 0.366488
[100]	training's rmse: 0.

[1380]	training's rmse: 0.179634	valid_1's rmse: 0.23209
[1390]	training's rmse: 0.17942	valid_1's rmse: 0.232042
[1400]	training's rmse: 0.17919	valid_1's rmse: 0.23198
[1410]	training's rmse: 0.178961	valid_1's rmse: 0.231912
[1420]	training's rmse: 0.178753	valid_1's rmse: 0.231872
[1430]	training's rmse: 0.178549	valid_1's rmse: 0.231831
[1440]	training's rmse: 0.178339	valid_1's rmse: 0.231766
[1450]	training's rmse: 0.178125	valid_1's rmse: 0.231731
[1460]	training's rmse: 0.177903	valid_1's rmse: 0.23169
[1470]	training's rmse: 0.17771	valid_1's rmse: 0.231629
[1480]	training's rmse: 0.177517	valid_1's rmse: 0.231597
[1490]	training's rmse: 0.177325	valid_1's rmse: 0.231547
[1500]	training's rmse: 0.177132	valid_1's rmse: 0.231517
[1510]	training's rmse: 0.176926	valid_1's rmse: 0.231468
[1520]	training's rmse: 0.176737	valid_1's rmse: 0.231424
[1530]	training's rmse: 0.176527	valid_1's rmse: 0.231402
[1540]	training's rmse: 0.176325	valid_1's rmse: 0.23137
[1550]	training's rms

[2800]	training's rmse: 0.155031	valid_1's rmse: 0.227059
[2810]	training's rmse: 0.154878	valid_1's rmse: 0.227028
[2820]	training's rmse: 0.154725	valid_1's rmse: 0.227001
[2830]	training's rmse: 0.154574	valid_1's rmse: 0.22697
[2840]	training's rmse: 0.154419	valid_1's rmse: 0.226944
[2850]	training's rmse: 0.154271	valid_1's rmse: 0.226919
[2860]	training's rmse: 0.15413	valid_1's rmse: 0.226892
[2870]	training's rmse: 0.153988	valid_1's rmse: 0.226871
[2880]	training's rmse: 0.153846	valid_1's rmse: 0.226848
[2890]	training's rmse: 0.153695	valid_1's rmse: 0.226819
[2900]	training's rmse: 0.153546	valid_1's rmse: 0.226792
[2910]	training's rmse: 0.153398	valid_1's rmse: 0.226771
[2920]	training's rmse: 0.153248	valid_1's rmse: 0.226748
[2930]	training's rmse: 0.153107	valid_1's rmse: 0.226721
[2940]	training's rmse: 0.152975	valid_1's rmse: 0.226689
[2950]	training's rmse: 0.152829	valid_1's rmse: 0.226655
[2960]	training's rmse: 0.152686	valid_1's rmse: 0.226623
[2970]	training'

[4220]	training's rmse: 0.137552	valid_1's rmse: 0.224342
[4230]	training's rmse: 0.137451	valid_1's rmse: 0.224331
[4240]	training's rmse: 0.137346	valid_1's rmse: 0.224326
[4250]	training's rmse: 0.137249	valid_1's rmse: 0.22431
[4260]	training's rmse: 0.137142	valid_1's rmse: 0.224294
[4270]	training's rmse: 0.13704	valid_1's rmse: 0.224274
[4280]	training's rmse: 0.136934	valid_1's rmse: 0.22426
[4290]	training's rmse: 0.136835	valid_1's rmse: 0.224242
[4300]	training's rmse: 0.136747	valid_1's rmse: 0.224237
[4310]	training's rmse: 0.136647	valid_1's rmse: 0.224233
[4320]	training's rmse: 0.136551	valid_1's rmse: 0.224223
[4330]	training's rmse: 0.136454	valid_1's rmse: 0.224212
[4340]	training's rmse: 0.136354	valid_1's rmse: 0.224196
[4350]	training's rmse: 0.136253	valid_1's rmse: 0.224184
[4360]	training's rmse: 0.136145	valid_1's rmse: 0.224171
[4370]	training's rmse: 0.136049	valid_1's rmse: 0.224167
[4380]	training's rmse: 0.135955	valid_1's rmse: 0.22415
[4390]	training's 

[5660]	training's rmse: 0.124763	valid_1's rmse: 0.223192
[5670]	training's rmse: 0.124684	valid_1's rmse: 0.223185
[5680]	training's rmse: 0.124608	valid_1's rmse: 0.223182
[5690]	training's rmse: 0.12454	valid_1's rmse: 0.223183
[5700]	training's rmse: 0.124473	valid_1's rmse: 0.223171
[5710]	training's rmse: 0.124394	valid_1's rmse: 0.223162
[5720]	training's rmse: 0.124325	valid_1's rmse: 0.223159
[5730]	training's rmse: 0.124253	valid_1's rmse: 0.22315
[5740]	training's rmse: 0.124171	valid_1's rmse: 0.223149
[5750]	training's rmse: 0.124094	valid_1's rmse: 0.223142
[5760]	training's rmse: 0.124021	valid_1's rmse: 0.22314
[5770]	training's rmse: 0.123939	valid_1's rmse: 0.223121
[5780]	training's rmse: 0.123861	valid_1's rmse: 0.223117
[5790]	training's rmse: 0.123779	valid_1's rmse: 0.223112
[5800]	training's rmse: 0.1237	valid_1's rmse: 0.223109
[5810]	training's rmse: 0.123622	valid_1's rmse: 0.22311
[5820]	training's rmse: 0.123546	valid_1's rmse: 0.223104
[5830]	training's rm

[7090]	training's rmse: 0.114753	valid_1's rmse: 0.222655
[7100]	training's rmse: 0.114689	valid_1's rmse: 0.222656
[7110]	training's rmse: 0.114625	valid_1's rmse: 0.222657
[7120]	training's rmse: 0.114567	valid_1's rmse: 0.222651
[7130]	training's rmse: 0.114507	valid_1's rmse: 0.222648
[7140]	training's rmse: 0.11444	valid_1's rmse: 0.222643
[7150]	training's rmse: 0.114378	valid_1's rmse: 0.22264
[7160]	training's rmse: 0.11432	valid_1's rmse: 0.222641
[7170]	training's rmse: 0.114259	valid_1's rmse: 0.222642
[7180]	training's rmse: 0.114196	valid_1's rmse: 0.222637
[7190]	training's rmse: 0.114136	valid_1's rmse: 0.22264
[7200]	training's rmse: 0.114074	valid_1's rmse: 0.222638
[7210]	training's rmse: 0.11401	valid_1's rmse: 0.222637
[7220]	training's rmse: 0.113951	valid_1's rmse: 0.222637
[7230]	training's rmse: 0.113891	valid_1's rmse: 0.222638
[7240]	training's rmse: 0.113831	valid_1's rmse: 0.222638
[7250]	training's rmse: 0.113767	valid_1's rmse: 0.222638
[7260]	training's r

In [8]:
"""
5.1:字符序列模型提取特征
?这里是字符序列，每个样本的长度，比词袋模型对应的长度小很多，所以可以运行。
"""
x_train, x_test, y_train, y_test = get_feature_charseq()
do_rnn(x_train, x_test, y_train, y_test)

Training Step: 1198  | total loss: 0.32488 | time: 7031.958s
| Adam | epoch: 001 | loss: 0.32488 - acc: 0.8838 -- iter: 11980/11989
Training Step: 1199  | total loss: 0.30527 | time: 7040.575s
| Adam | epoch: 001 | loss: 0.30527 - acc: 0.8954 | val_loss: 0.19297 - val_acc: 0.9334 -- iter: 11989/11989
--
0.81415826
0.028384151
0.028390449
0.9462246
0.046222437
0.90763736
0.028523827
0.05547835
0.9446823
0.028656604
0.029476726
0.0279688
0.029103251
0.03055239
0.030737001
0.6035332
0.028961804
0.9447098
0.932022
0.80901957
0.027427709
0.9353615
0.3105995
0.028429676
0.6321551
0.027613789
0.030010343
0.93016225
0.69819474
0.031622317
0.02844428
0.041175142
0.94100815
0.028489834
0.94654024
0.94388837
0.9456376
0.9363762
0.9481235
0.027980527
0.6263876
0.043506738
0.028271616
0.0277728
0.94908077
0.9153437
0.031730775
0.0927457
0.89220446
0.029739736
0.8342079
0.029343098
0.81613433
0.82037216
0.93141836
0.8900175
0.02889042
0.032855876
0.50485474
0.75830084
0.92102647
0.035015784
0.026604

0.2849521
0.8966595
0.028749594
0.94748235
0.7855638
0.053528115
0.913151
0.9372193
0.44183657
0.027919935
0.9274479
0.18528117
0.86441267
0.6808073
0.69175404
0.9498872
0.9505656
0.028617462
0.18993913
0.75957674
0.61815965
0.031713806
0.9398293
0.8860182
0.9421261
0.949101
0.037592113
0.027665202
0.9387481
0.02783886
0.6875324
0.027237428
0.027870417
0.034014005
0.952491
0.030512001
0.92921525
0.033460036
0.9267372
0.81689596
0.8557101
0.7094066
0.8704785
0.9143147
0.027792659
0.21464227
0.93633914
0.9523878
0.89032376
0.02768819
0.03484134
0.70991427
0.93871206
0.051584113
0.8631573
0.066188104
0.9252504
0.94362485
0.026850313
0.94460607
0.9447723
0.031570535
0.049687818
0.905713
0.39998063
0.035069197
0.0287927
0.47969237
0.031942602
0.37725475
0.026741505
0.030239118
0.02778357
0.36833042
0.28228432
0.9427773
0.031002607
0.8659497
0.92564213
0.87643844
0.02822567
0.029908134
0.030849798
0.02825347
0.19606398
0.94653106
0.94958454
0.041267518
0.027405731
0.3947862
0.942861
0.715425

0.9096511
0.028928524
0.8580842
0.25976413
0.9296368
0.77409106
0.24241409
0.8226177
0.95254266
0.68820155
0.9517
0.94750416
0.040935367
0.552129
0.8396038
0.21937397
0.9497345
0.02909731
0.94658756
0.02927366
0.8590254
0.028045524
0.48600256
0.030727083
0.5205682
0.9399401
0.94011855
0.030786945
0.028095173
0.030185867
0.028463505
0.946973
0.76448524
0.45384353
0.041846197
0.42461458
0.9275469
0.8102605
0.91684675
0.6647399
0.043532845
0.8730884
0.03722134
0.45914078
0.9247747
0.917032
0.9241969
0.9517652
0.028468274
0.027811283
0.026889622
0.81663984
0.031103477
0.027899604
0.89493823
0.9431553
0.029777076
0.026462832
0.73365223
0.6192545
0.74331397
0.06411814
0.93091124
0.02714973
0.026786717
0.9170293
0.028181722
0.94946235
0.9460727
0.07009656
0.108687066
0.029385148
0.029823199
0.47551355
0.029104518
0.9041336
0.027654244
0.02732993
0.031192398
0.04885428
0.027255261
0.94840866
0.94606787
0.02902328
0.027386706
0.8887535
0.045958232
0.41700602
0.9426937
0.48989144
0.051583495
0.9

0.075900696
0.9462716
0.029013643
0.028353931
0.0298421
0.93806326
0.028208174
0.90145457
0.91470057
0.9405354
0.029478809
0.036292084
0.9482295
0.9464846
0.030948283
0.03493218
0.03307917
0.949304
0.04235698
0.93154144
0.59457314
0.027568504
0.028147163
0.9100666
0.9441374
0.027885964
0.9514558
0.559316
0.027845696
0.5701147
0.028446553
0.027345048
0.83963996
0.03215407
0.6968839
0.94460946
0.9473766
0.90314746
0.94771624
0.7070397
0.02923363
0.069356844
0.8862281
0.9452885
0.0269289
0.54827094
0.028865682
0.030093495
0.029606342
0.56650317
0.029078381
0.8828896
0.6448903
0.02828496
0.942376
0.030675853
0.03832062
0.030712694
0.031242669
0.932305
0.02974209
0.027575348
0.043830536
0.034581807
0.90902674
0.15263373
0.028946387
0.045272525
0.9082983
0.93724376
0.94722956
0.030947452
0.31970358
0.028730202
0.94895303
0.03711576
0.038072895
0.15867208
0.030639268
0.027405402
0.031306446
0.94059014
0.63732713
0.17167838
0.028328666
0.027744379
0.82640785
0.90797424
0.8907364
0.027035397
0.

0.03161145
0.4216015
0.74322104
0.055226658
0.94128966
0.9409492
0.029606987
0.030325083
0.029370926
0.9441303
0.027137822
0.028253645
0.027892608
0.9035456
0.9476963
0.5691856
0.62982404
0.94675016
0.03698854
0.02973074
0.029968884
0.654434
0.035501655
0.02876465
0.08769411
0.032262202
0.88486546
0.033932664
0.035989616
0.9295938
0.87472576
0.3192538
0.95203567
0.030033847
0.9491784
0.93752134
0.03214907
0.9496524
0.8367438
0.028607195
0.89772266
0.26612034
0.030541573
0.6019441
0.95195454
0.9093455
0.027521363
0.03486078
0.45131934
0.86479443
0.048142776
0.19095132
0.027884701
0.027364912
0.034165137
0.9513873
0.06849967
0.027152505
0.65433955
0.032254476
0.029488578
0.029410103
0.8176526
0.95010036
0.3179661
0.5131494
0.032040548
0.15737799
0.94867325
0.9352939
0.9417299
0.80424166
0.8381742
0.02729099
0.2421944
0.9461243
0.02997499
0.95179456
0.94001055
0.028849898
0.90825284
0.8354932
0.73562145
0.02811603
0.94863635
0.044511005
0.9455116
0.9001985
0.030262183
0.18409522
0.0300000

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      3910
           1       0.94      0.93      0.93      4083

    accuracy                           0.93      7993
   macro avg       0.93      0.93      0.93      7993
weighted avg       0.93      0.93      0.93      7993

[[3668  242]
 [ 290 3793]]
